In [7]:
import pygrib

grbs = pygrib.open('temp')

In [8]:
grb = grbs.select(name='Temperature')[-1]

In [69]:
grb.data()[0].data[180,0]

201.71119140625

In [68]:
grb.data(lat1=0,lat2=0,lon1=0,lon2=0)

(masked_array(data=[[201.71119140625]],
              mask=[[False]],
        fill_value=9999.0),
 array([[0.]]),
 array([[0.]]))

In [3]:
for grb in grbs:
    print(grb)

1:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
2:Temperature:K (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
3:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
4:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
5:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 0 hrs:from 202311050600:ens mean
6:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
7:Temperature:K (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
8:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 0 hrs:from 202311050600:ens mean
9:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 5000 

In [3]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from datetime import datetime
import time
import pygrib
import pandas as pd

ct_df = pd.read_csv("uscities.csv")[:10]
lls = list(zip(ct_df['lat'], ct_df['lng']))

dt = datetime.now()
file_dt = "".join(list(map(str,[dt.year,dt.month,f"{dt.day:02d}"])))

client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = client.list_objects(Bucket='noaa-gefs-pds', Prefix = "gefs."+file_dt+"/", Delimiter="/",)

while True:
    if bucket.get('CommonPrefixes') is not None:
        fldr = bucket.get('CommonPrefixes')[-1].get('Prefix')
        break
    else:
        time.sleep(300)

cycle = fldr.split('/')[-2]
base = f'{fldr[:-1]}/atmos/pgrb2ap5/gec00.t{cycle}z.pgrb2a.0p50.f'
files = []
for i in range(0,243,3):
    files.append(base+f"{i:03d}")
for i in range(246,390,6):
    files.append(base+f"{i}")


def process(file, cycle):
    grbs = pygrib.open(file)
    grb = grbs.select(name='Temperature')[-1]
    data = grb.data()[0]
    
    def f(lat, lon):
        return round(abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65),2)
    
    ct_df['dd_'+cycle] = [f(lat, lon) for lat,lon in lls]
    
    grbs.close()

while files:
    idx = int(files[0].split(".")[-1][-3:])
    print(files[0])
    try:
        if idx%24==0 and idx>0:
            ct_df['dd'] = (ct_df['dd_00']+ct_df['dd_03']+ct_df['dd_06']+ct_df['dd_09']+ct_df['dd_12']+ct_df['dd_15']+ct_df['dd_18']+ct_df['dd_21'])/8
            break

        client.download_file('noaa-gefs-pds', files[0], 'temp')
        process('temp', f"{idx%24:02d}")
        
    except Exception as e:
        print(e)
        # time.sleep(60)
        break
    del files[0]


gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f000
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f003
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f006
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f009
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f012
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f015
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f018
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f021
gefs.20231105/12/atmos/pgrb2ap5/gec00.t12z.pgrb2a.0p50.f024


In [6]:
ct_df.groupby('state_name').apply(lambda x: x['population']*x['dd'])

,county_fips,lat,lng,population,density,military,incorporated,ranking,id,dd_00,dd_03,dd_06,dd_09,dd_12,dd_15,dd_18,dd_21,dd
state_name,,,,,,,,,,,,,,,,,,
California,6037.0,34.11410,-118.4068,12121244.0,3267.6,0.0,1.0,1.0,1.840020e+09,0.08,0.59,4.73,7.35,5.99,3.29,2.88,1.94,3.35625
District of Columbia,11001.0,38.90470,-77.0163,4810669.0,4434.0,0.0,1.0,1.0,1.840006e+09,12.52,7.15,1.03,0.93,5.35,10.57,14.04,15.70,8.41125
Florida,12086.0,25.78400,-80.2101,5711945.0,4945.7,0.0,1.0,1.0,1.840015e+09,8.00,8.15,9.95,8.97,7.97,8.33,7.02,6.08,8.05875
Georgia,13121.0,33.76280,-84.4220,5046555.0,1419.9,0.0,1.0,1.0,1.840014e+09,11.44,3.55,5.45,7.71,1.13,3.01,6.48,7.96,5.84125
Illinois,17031.0,41.83750,-87.6866,8595181.0,4576.6,0.0,1.0,1.0,1.840000e+09,17.92,16.33,11.47,7.95,8.41,9.49,9.54,9.40,11.31375
Massachusetts,25025.0,42.31880,-71.0852,4208580.0,5505.8,0.0,1.0,1.0,1.840000e+09,15.58,12.55,10.57,9.75,12.19,17.59,21.60,23.08,15.36375
New York,36081.0,40.69430,-73.9249,18972871.0,10768.2,0.0,1.0,1.0,1.840034e+09,11.80,11.29,8.41,6.15,8.59,14.35,16.56,19.12,12.03375
Pennsylvania,42101.0,40.00770,-75.1339,5512873.0,4544.9,0.0,1.0,1.0,1.840001e+09,14.14,10.75,5.35,4.35,8.59,15.25,17.46,19.30,11.89875
Texas,48157.0,31.28975,-96.0776,5659537.5,1458.4,0.0,1.0,1.0,1.840020e+09,1.53,1.22,9.32,12.39,9.59,6.71,5.22,4.37,6.29375


In [8]:
ct_df["lol"] = [11.80007812499997, 0.07992187500002501, 17.920078124999975, 7.999921874999984, 2.8000781249999704, 0.259921874999975, 14.140078125000002, 11.44007812500007, 12.520078124999998, 15.580078124999943]

In [9]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,lol
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016,11.800078
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491,0.079922
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,17.920078
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149,7.999922
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440,2.800078
5,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5650910,1394.6,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925,0.259922
6,Philadelphia,Philadelphia,PA,Pennsylvania,42101,Philadelphia,40.0077,-75.1339,5512873,4544.9,shape,False,True,America/New_York,1,19154 19151 19150 19153 19152 19102 19103 1910...,1840000673,14.140078
7,Atlanta,Atlanta,GA,Georgia,13121,Fulton,33.7628,-84.4220,5046555,1419.9,shape,False,True,America/New_York,1,30334 30331 30332 30309 30308 30303 30354 3032...,1840013660,11.440078
8,Washington,Washington,DC,District of Columbia,11001,District of Columbia,38.9047,-77.0163,4810669,4434.0,shape,False,True,America/New_York,1,20250 20010 20011 20012 20015 20520 20064 2041...,1840006060,12.520078
9,Boston,Boston,MA,Massachusetts,25025,Suffolk,42.3188,-71.0852,4208580,5505.8,shape,False,True,America/New_York,1,02120 02121 02122 02124 02125 02126 02127 0212...,1840000455,15.580078


In [9]:
lls = zip(ct_df['lat'], ct_df['lng'])
data = grb.data()[0]

def f(lat, lon):
    print(data[int(round(-2*lat)+180),int(round(2*lon))])
    return abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65)

ct_df['dd_00'] = [f(lat, lon) for lat,lon in lls]

284.927734375
291.527734375
281.527734375
295.927734375
289.927734375
291.627734375
283.627734375
285.127734375
284.527734375
282.82773437500003


In [10]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,dd_00
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016,11.800078
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491,0.079922
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,17.920078
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149,7.999922
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440,2.800078
5,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5650910,1394.6,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925,0.259922
6,Philadelphia,Philadelphia,PA,Pennsylvania,42101,Philadelphia,40.0077,-75.1339,5512873,4544.9,shape,False,True,America/New_York,1,19154 19151 19150 19153 19152 19102 19103 1910...,1840000673,14.140078
7,Atlanta,Atlanta,GA,Georgia,13121,Fulton,33.7628,-84.4220,5046555,1419.9,shape,False,True,America/New_York,1,30334 30331 30332 30309 30308 30303 30354 3032...,1840013660,11.440078
8,Washington,Washington,DC,District of Columbia,11001,District of Columbia,38.9047,-77.0163,4810669,4434.0,shape,False,True,America/New_York,1,20250 20010 20011 20012 20015 20520 20064 2041...,1840006060,12.520078
9,Boston,Boston,MA,Massachusetts,25025,Suffolk,42.3188,-71.0852,4208580,5505.8,shape,False,True,America/New_York,1,02120 02121 02122 02124 02125 02126 02127 0212...,1840000455,15.580078


In [51]:
grb.data(lat1=34, lat2=34.5, lon1=241.5, lon2=242)

(array([[297. , 298.8],
        [292.7, 294.9]]),
 array([[34.5, 34.5],
        [34. , 34. ]]),
 array([[241.5, 242. ],
        [241.5, 242. ]]))

In [20]:
ct_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id,dd_00
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18972871,10768.2,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016,12.75
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12121244,3267.6,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491,2.19
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8595181,4576.6,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494,17.79
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,5711945,4945.7,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149,9.39
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5668165,1522.2,shape,False,True,America/Chicago,1,75287 75098 75234 75254 75251 75252 75253 7503...,1840019440,2.49
5,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5650910,1394.6,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925,0.21
6,Philadelphia,Philadelphia,PA,Pennsylvania,42101,Philadelphia,40.0077,-75.1339,5512873,4544.9,shape,False,True,America/New_York,1,19154 19151 19150 19153 19152 19102 19103 1910...,1840000673,14.37
7,Atlanta,Atlanta,GA,Georgia,13121,Fulton,33.7628,-84.4220,5046555,1419.9,shape,False,True,America/New_York,1,30334 30331 30332 30309 30308 30303 30354 3032...,1840013660,11.13
8,Washington,Washington,DC,District of Columbia,11001,District of Columbia,38.9047,-77.0163,4810669,4434.0,shape,False,True,America/New_York,1,20250 20010 20011 20012 20015 20520 20064 2041...,1840006060,13.11
9,Boston,Boston,MA,Massachusetts,25025,Suffolk,42.3188,-71.0852,4208580,5505.8,shape,False,True,America/New_York,1,02120 02121 02122 02124 02125 02126 02127 0212...,1840000455,16.35
